In [172]:
from sklearn.datasets import load_boston
!pip install optuna

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb
import xgboost as xgb
# sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

import sklearn.metrics

from mlxtend.classifier import StackingCVClassifier
!pip install vecstack
from vecstack import stacking

from itertools import combinations 


import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
sns.set()

In [192]:
X, y = load_boston(return_X_y=True)

In [191]:
def scaler_fuc(scaler):
  train_x, valid_x, train_y, valid_y = train_test_split(X, y,
                                              test_size=0.25, random_state = 123)
  if (scaler == 'minmax'):
    scaler = MinMaxScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    scaler.fit(valid_x)
    valid_x = scaler.transform(valid_x)

  if (scaler == 'stand'):
    scaler = StandardScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    scaler.fit(valid_x)
    valid_x = scaler.transform(valid_x)

  if (scaler == 'log'):
    transformer = FunctionTransformer(np.log1p, validate=True)
    train_x = transformer.transform(train_x)
    valid_x = transformer.transform(valid_x)

  # turning these train/tests into lgb/xgb datasets
  dtrain_gbm = lgb.Dataset(train_x, label=train_y)
  dvalid_gbm = lgb.Dataset(valid_x, label=valid_y)

  dtrain_xbg = xgb.DMatrix(train_x, label=train_y)
  dvalid_xbg = xgb.DMatrix(valid_x, label=valid_y)
  return train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg

In [194]:
# models = ['lreg', 'las', 'rid','ada', 'et', 'rf', 'xgb', 'gbm'] # when you want to try all of them/ iteration: 1 
models = ['ada', 'et', 'rf', 'xgb', 'gbm'] # selecting the 5 best models from iteration 1 (by observing the log) / iteration: 2
comb = combinations(models, 3) 
comb = list(comb)

In [195]:

class Objective:

    def __init__(self):
        self.best_gbm = None
        self._gbm = None
        self.best_xgb = None
        self._xgb = None
        self.predictions = None
        self.fpredictions = None

    def __call__(self, trial):

        i = trial.suggest_int("combos", 0, (len(comb)-1))
 
        gbm_preds = 0
        gbm_rmse = float('inf')
        xgb_preds = 0
        xgb_rmse = float('inf')
        rf_preds = 0
        rf_rmse = float('inf')
        et_preds = 0
        et_rmse = float('inf')
        ada_preds = 0
        ada_rmse = float('inf')
        lreg_preds = 0
        lreg_rmse = float('inf')
        las_preds = 0
        las_rmse = float('inf')
        rid_preds = 0
        rid_rmse = float('inf')

        ###############################################################################
        #                                 . Scaling                                   #
        ###############################################################################
        # If you want 1 scaled data for everything use this, other wise leave this commented
        # scaler = trial.suggest_categorical("Scaler", ['minmax','stand','log'])
        # train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(scaler)
        ###############################################################################
        #                            . Linear Regression                              #
        ###############################################################################
        if any(x == 'lreg' for x in comb[i]):
          lr_scaler = trial.suggest_categorical("lr_Scaler", ['minmax','stand','log'])
          train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(lr_scaler)
          lreg = LinearRegression().fit(train_x, train_y)
          lreg_preds = lreg.predict(valid_x)
          lreg_rmse = sklearn.metrics.mean_squared_error(valid_y, lreg_preds, squared = False)
        ###############################################################################
        #                                 . Lasso                                     #
        ###############################################################################
        if any(x == 'las' for x in comb[i]):
          las_scaler = trial.suggest_categorical("las_Scaler", ['minmax','stand','log'])
          train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(las_scaler)
          alphal = trial.suggest_loguniform("alphal", 1e-8, 1.0)
          las = linear_model.Lasso(alpha=alphal).fit(train_x, train_y)
          las_preds = las.predict(valid_x)
          las_rmse = sklearn.metrics.mean_squared_error(valid_y, las_preds, squared = False)
        ###############################################################################
        #                                 . Ridge                                     #
        ###############################################################################
        if any(x == 'rid' for x in comb[i]):
          rid_scaler = trial.suggest_categorical("rid_Scaler", ['minmax','stand','log'])
          train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(rid_scaler)
          alphar = trial.suggest_loguniform("alphar", 1e-8, 1.0)
          rid = Ridge(alpha=alphar).fit(train_x, train_y)
          rid_preds = rid.predict(valid_x)
          rid_rmse = sklearn.metrics.mean_squared_error(valid_y, rid_preds, squared = False)
        ###############################################################################
        #                               . Ada Boost                                   #
        ###############################################################################
        if any(x == 'ada' for x in comb[i]):
          ada_scaler = trial.suggest_categorical("ada_Scaler", ['minmax','stand','log'])
          train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(ada_scaler)
          ada_ne = trial.suggest_int("ada_ne", 1, 1000)
          ada_lr = trial.suggest_loguniform("ada_lr", 0.05, 1)
          ada = AdaBoostRegressor(n_estimators = ada_ne, learning_rate = ada_lr,
                                     random_state=0).fit(train_x, train_y)
          ada_preds = ada.predict(valid_x)
          ada_rmse = sklearn.metrics.mean_squared_error(valid_y, ada_preds, squared = False)
        ###############################################################################
        #                              . Extra Trees                                  #
        ###############################################################################
        if any(x == 'et' for x in comb[i]):
          et_scaler = trial.suggest_categorical("et_Scaler", ['minmax','stand','log'])
          train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(et_scaler)
          et_md = trial.suggest_int("et_max_depth", 1, 20)
          et_ne = trial.suggest_int("et_ne", 1, 1000)
          et = ExtraTreesRegressor(max_depth=et_md, n_estimators = et_ne,
                                     random_state=0).fit(train_x, train_y)
          et_preds = et.predict(valid_x)
          et_rmse = sklearn.metrics.mean_squared_error(valid_y, et_preds, squared = False)
        ###############################################################################
        #                             . Random Forest                                 #
        ###############################################################################
        if any(x == 'rf' for x in comb[i]):
          rf_scaler = trial.suggest_categorical("rf_Scaler", ['minmax','stand','log'])
          train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(rf_scaler)
          rf_md = trial.suggest_int("rf_max_depth", 1, 20)
          rf_ne = trial.suggest_int("rf_ne", 1, 1000)
          rf = RandomForestRegressor(max_depth=rf_md, n_estimators = rf_ne,
                                     random_state=0).fit(train_x, train_y)
          rf_preds = rf.predict(valid_x)
          rf_rmse = sklearn.metrics.mean_squared_error(valid_y, rf_preds, squared = False)
        ###############################################################################
        #                                 . XGBoost                                   #
        ###############################################################################
        if any(x == 'xgb' for x in comb[i]): 
          xgb_scaler = trial.suggest_categorical("xgb_Scaler", ['minmax','stand','log'])
          train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(xgb_scaler)
          xgb_param = {
              "silent": 1,
              "objective": "reg:squarederror",
              "eval_metric": "rmse",
              "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
              "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
              "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
          }
          if xgb_param["booster"] == "gbtree" or xgb_param["booster"] == "dart":
              xgb_param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
              xgb_param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
              xgb_param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
              xgb_param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
          if xgb_param["booster"] == "dart":
              xgb_param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
              xgb_param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
              xgb_param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
              xgb_param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)

          # Add a callback for pruning.
          xgb_pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse" )
          xgb_ = xgb.train(xgb_param, dtrain_xbg, evals=[(dvalid_xbg, "validation")], callbacks=[xgb_pruning_callback])
          xgb_preds = xgb_.predict(dvalid_xbg)
          xgb_rmse = sklearn.metrics.mean_squared_error(valid_y, xgb_preds, squared = False)
          self._xgb = xgb_
        ###############################################################################
        #                          . Light Gradient Boosting                          #
        ###############################################################################
        if any(x == 'gbm' for x in comb[i]):
          gbm_scaler = trial.suggest_categorical("gbm_Scaler", ['minmax','stand','log'])
          train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(gbm_scaler)
          gbm_param = {
            'objective': 'regression',
            'metric': 'rmse',
              "verbosity": -1,
              "boosting_type": "gbdt",
              "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10), 
              "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10),
              "num_leaves": trial.suggest_int("num_leaves", 2, 256), 
              "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0), 
              "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
              "bagging_freq": trial.suggest_int("bagging_freq", 1, 7), 
              "min_child_samples": trial.suggest_int("min_child_samples", 2, 100), 
          }
          # Add a callback for pruning.
          gbm_pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
          gbm = lgb.train(gbm_param, dtrain_gbm, valid_sets=[dvalid_gbm], verbose_eval=False, callbacks=[gbm_pruning_callback])
          gbm_preds = gbm.predict(valid_x)
          gbm_rmse = sklearn.metrics.mean_squared_error(valid_y, gbm_preds, squared = False)
          self._gbm = gbm
        ###############################################################################
        #                            . Stacking Strategy                              #
        ###############################################################################

        # strat = trial.suggest_int("strat", 0, 1)

        # if (strat == 0): # taking mean
        preds = (gbm_preds + xgb_preds + rf_preds + \
        las_preds + ada_preds + et_preds + \
        lreg_preds + rid_preds) / 3


        # if (strat == 1): # Making an equation
        # a = trial.suggest_uniform("a", 0.1, 0.9)
        # b = trial.suggest_uniform("b", 0.1, 0.9)
        # c = trial.suggest_uniform("c", 0.1, 0.9)
        # d = trial.suggest_uniform("d", 0.1, 0.9)

        # def custom_sort(t):
        #   return t[0]

        # R = [[gbm_rmse,gbm_preds] , [xgb_rmse,xgb_preds], [rf_rmse, rf_preds], \
        #     [las_rmse,las_preds], [ada_rmse,ada_preds],[et_rmse,et_preds],\
        #     [lreg_rmse,lreg_preds], [rid_rmse,rid_preds]]
        # R.sort(key=custom_sort) # low to high
        # m1 = R[0][1]
        # m2 = R[1][1]
        # m3 = R[2][1]
        # # Using list comprehension. This may result in a divide by 0 error when rounding occurs.
        # # A fix would be to use a more precise library
        # preds = [ (a*((x**b) + (y**c))) + (d*z) for x,y,z in zip(m1,m2,m3) ]
        self.predictions = preds
        rmse = sklearn.metrics.mean_squared_error(valid_y, preds, squared = False)
        return rmse

    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_gbm = self._gbm
            self.best_xgb = self._xgb
            self.fpredictions = self.predictions

In [196]:
import optuna
objective = Objective()

# Setting SEED 
from optuna.samplers import TPESampler
sampler = TPESampler(seed=10)

study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize",
    sampler=sampler
)
study.optimize(objective, n_trials=1000, callbacks=[objective.callback])

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

best_gbm = objective.best_gbm
best_xgb = objective.best_xgb

[0]	validation-rmse:8.5838
[1]	validation-rmse:7.51008
[2]	validation-rmse:7.07943
[3]	validation-rmse:6.71247
[4]	validation-rmse:6.44665
[5]	validation-rmse:6.23397
[6]	validation-rmse:6.06656
[7]	validation-rmse:5.93394
[8]	validation-rmse:5.82863
[9]	validation-rmse:5.73827


[I 2020-07-08 13:32:06,592] Finished trial#0 with value: 3.975394457608443 with parameters: {'combos': 9, 'rf_Scaler': 'stand', 'rf_max_depth': 5, 'rf_ne': 528, 'xgb_Scaler': 'minmax', 'booster': 'gblinear', 'lambda': 0.00978207662259244, 'alpha': 9.728728830009641e-05, 'gbm_Scaler': 'minmax', 'lambda_l1': 1.684789145169785e-08, 'lambda_l2': 0.024147495209786127, 'num_leaves': 75, 'feature_fraction': 0.5014665019375213, 'bagging_fraction': 0.4530038885044062, 'bagging_freq': 5, 'min_child_samples': 18}. Best is trial#0 with value: 3.975394457608443.
[I 2020-07-08 13:32:07,673] Finished trial#1 with value: 4.186402570791736 with parameters: {'combos': 4, 'ada_Scaler': 'log', 'ada_ne': 345, 'ada_lr': 0.5704456208852126, 'rf_Scaler': 'log', 'rf_max_depth': 2, 'rf_ne': 201, 'gbm_Scaler': 'minmax', 'lambda_l1': 4.235304245072407e-06, 'lambda_l2': 1.8195630232649103, 'num_leaves': 79, 'feature_fraction': 0.47887489144151374, 'bagging_fraction': 0.6482004246510716, 'bagging_freq': 7, 'min_chi

[0]	validation-rmse:8.48526
[1]	validation-rmse:7.78859
[2]	validation-rmse:7.49223
[3]	validation-rmse:7.34265
[4]	validation-rmse:7.24649
[5]	validation-rmse:7.17212
[6]	validation-rmse:7.10849
[7]	validation-rmse:7.05055
[8]	validation-rmse:6.99575
[9]	validation-rmse:6.94273


[I 2020-07-08 13:32:08,998] Finished trial#2 with value: 3.872957306592403 with parameters: {'combos': 6, 'et_Scaler': 'log', 'et_max_depth': 13, 'et_ne': 322, 'rf_Scaler': 'stand', 'rf_max_depth': 5, 'rf_ne': 284, 'xgb_Scaler': 'log', 'booster': 'gblinear', 'lambda': 0.0006431490422420297, 'alpha': 0.027655758533508757}. Best is trial#2 with value: 3.872957306592403.


[0]	validation-rmse:23.9807
[1]	validation-rmse:23.9807
[2]	validation-rmse:23.9807
[3]	validation-rmse:23.9807
[4]	validation-rmse:23.9807
[5]	validation-rmse:23.9807
[6]	validation-rmse:23.9807
[7]	validation-rmse:23.9807
[8]	validation-rmse:23.9807
[9]	validation-rmse:23.9807


...

[0]	validation-rmse:16.7177
[1]	validation-rmse:11.913
[2]	validation-rmse:8.68823
[3]	validation-rmse:6.62358
[4]	validation-rmse:5.4267
[5]	validation-rmse:4.64771
[6]	validation-rmse:4.25325
[7]	validation-rmse:3.93473
[8]	validation-rmse:3.87738
[9]	validation-rmse:3.84108


[I 2020-07-08 14:12:45,974] Finished trial#998 with value: 3.409376882490111 with parameters: {'combos': 1, 'ada_Scaler': 'log', 'ada_ne': 82, 'ada_lr': 0.4516902166384415, 'et_Scaler': 'stand', 'et_max_depth': 9, 'et_ne': 627, 'xgb_Scaler': 'log', 'booster': 'gbtree', 'lambda': 0.04273169015828022, 'alpha': 0.011128749469161547, 'max_depth': 3, 'eta': 0.3108971975110843, 'gamma': 3.1973109896856295e-08, 'grow_policy': 'depthwise'}. Best is trial#881 with value: 3.06716894171875.


[0]	validation-rmse:7.84421
[1]	validation-rmse:4.28042
[2]	validation-rmse:4.07671
[3]	validation-rmse:4.15154
[4]	validation-rmse:4.18091
[5]	validation-rmse:4.21876
[6]	validation-rmse:4.25333
[7]	validation-rmse:4.27839
[8]	validation-rmse:4.32215
[9]	validation-rmse:4.3593


[I 2020-07-08 14:12:47,030] Finished trial#999 with value: 3.3865367961676656 with parameters: {'combos': 1, 'ada_Scaler': 'log', 'ada_ne': 83, 'ada_lr': 0.6183807125477403, 'et_Scaler': 'stand', 'et_max_depth': 8, 'et_ne': 516, 'xgb_Scaler': 'log', 'booster': 'gbtree', 'lambda': 0.1189663592390239, 'alpha': 0.056581492422171385, 'max_depth': 5, 'eta': 0.7029625310165433, 'gamma': 6.974249033208561e-08, 'grow_policy': 'depthwise'}. Best is trial#881 with value: 3.06716894171875.


Best trial:
  Params: 
    combos: 1
    ada_Scaler: log
    ada_ne: 173
    ada_lr: 0.6716180264894074
    et_Scaler: stand
    et_max_depth: 10
    et_ne: 593
    xgb_Scaler: log
    booster: gbtree
    lambda: 0.056298103046901415
    alpha: 0.0005885345642231615
    max_depth: 3
    eta: 0.5865796691943621
    gamma: 6.697097340043158e-08
    grow_policy: depthwise


In [197]:
comb[1]

('ada', 'et', 'xgb')

3.067 : ('ada', 'et', 'xgb')



```
  Params: 
    combos: 1
    ada_Scaler: log
    ada_ne: 173
    ada_lr: 0.6716180264894074
    et_Scaler: stand
    et_max_depth: 10
    et_ne: 593
    xgb_Scaler: log
    booster: gbtree
    lambda: 0.056298103046901415
    alpha: 0.0005885345642231615
    max_depth: 3
    eta: 0.5865796691943621
    gamma: 6.697097340043158e-08
    grow_policy: depthwise
```





In [198]:
objective.fpredictions 

array([27.85772216, 27.39908626, 48.17995478, 18.33345253, 30.14070563,
       37.29053558, 26.25861886, 10.15475066, 18.14889132, 29.62243171,
       23.97859444, 19.98696824, 15.73101726, 26.59151738, 18.75253067,
       20.55269833, 20.15853826, 42.75472066, 18.58739498, 15.48510911,
       16.54584927, 45.3914744 , 35.21042146, 46.35623061, 48.51340702,
       21.79671212, 16.1662488 , 20.632855  , 20.63631538, 21.8016087 ,
       25.60592866, 31.17305858, 13.99109611, 21.13797821, 23.39973703,
       32.81045667, 24.95684238, 14.94596067, 16.64715835, 47.23880925,
       28.05968523, 19.49358329, 26.5581701 , 48.90793117, 16.9481093 ,
       23.05092158, 19.3891282 , 22.99633326, 17.07364299, 19.91790302,
       30.94912602, 25.8317543 , 19.39997722, 10.88814199, 20.99495711,
       16.48914731, 13.93787858, 10.18333297, 33.5668052 , 11.20237941,
       19.91155235, 18.3737015 , 16.89461203, 19.29234017, 21.27131705,
       24.04901804, 24.36425727, 20.89394213, 24.4201314 , 27.94